Copyright (c) Microsoft Corporation.

Licensed under the MIT License.

# Text2SQL with AutoGen & Azure OpenAI

This notebook demonstrates how the AutoGen Agents can be integrated with Azure OpenAI to answer questions from the database based on the schemas provided. 

A multi-shot approach is used for SQL generation for more reliable results and reduced token usage. More details can be found in the README.md.

### Dependencies

To install dependencies for this demo:

`uv sync --package autogen_text_2_sql`

`uv add --editable ../text_2_sql_core/`

In [1]:
# This is only needed for this notebook to work
import sys
from pathlib import Path

# Add the parent directory of `src` to the path
sys.path.append(str(Path.cwd() / "src"))

In [2]:
import dotenv
import logging
from autogen_agentchat.task import Console
from autogen_text_2_sql.autogen_text_2_sql import AutoGenText2Sql

In [3]:
logging.basicConfig(level=logging.INFO)

In [4]:
dotenv.load_dotenv()

True

## Bot setup

In [5]:
agentic_text_2_sql = AutoGenText2Sql(target_engine="TSQL", engine_specific_rules="Use TOP X to limit the number of rows returned instead of LIMIT X. NEVER USE LIMIT X as it produces a syntax error.").agentic_flow

In [6]:
result = agentic_text_2_sql.run_stream(task="What are the total number of sales within 2008?")

In [7]:
await Console(result)

INFO:autogen_core:Sending message of type GroupChatStart to group_chat_manager: {'message': TextMessage(source='user', models_usage=None, content='What are the total number of sales within 2008?')}
INFO:autogen_core:Calling message handler for group_chat_manager/6d7e11b2-73d2-4b4d-9716-dd5596ded914 with message type GroupChatStart sent by Unknown
INFO:autogen_core:Publishing message of type GroupChatStart to all subscribers: {'message': TextMessage(source='user', models_usage=None, content='What are the total number of sales within 2008?')}
INFO:autogen_core:Publishing message of type GroupChatStart to all subscribers: {'message': TextMessage(source='user', models_usage=None, content='What are the total number of sales within 2008?')}
INFO:root:Messages: [TextMessage(source='user', models_usage=None, content='What are the total number of sales within 2008?')]
INFO:root:Decision: sql_query_cache_agent
INFO:autogen_core:Publishing message of type GroupChatRequestPublish to all subscriber

---------- user ----------
What are the total number of sales within 2008?


INFO:httpx:HTTP Request: POST https://aoai-text2sql-adi.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-03-15-preview "HTTP/1.1 200 OK"
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://aisearch-text2sql-adi.search.windows.net/indexes('text-2-sql-query-cache-index')/docs/search.post.search?api-version=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '21275'
    'api-key': 'REDACTED'
    'Accept': 'application/json;odata.metadata=none'
    'x-ms-client-request-id': '6faf61f0-b0f1-11ef-8373-0242ac110002'
    'User-Agent': 'azsdk-python-search-documents/11.6.0b8 Python/3.12.7 (Linux-5.15.153.1-microsoft-standard-WSL2-x86_64-with-glibc2.36)'
A body is sent with the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/json; odata.metadata=none; odata.s

---------- sql_query_cache_agent ----------
{"contains_pre_run_results": false, "cached_questions_and_schemas": null}


INFO:httpx:HTTP Request: POST https://aoai-text2sql-adi.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
INFO:autogen_core.events:{"prompt_tokens": 178, "completion_tokens": 20, "type": "LLMCall"}
/workspaces/dstoolkit-text2sql-and-imageprocessing/.venv/lib/python3.12/site-packages/autogen_agentchat/agents/_assistant_agent.py:239: UserWarning: Resolved model mismatch: gpt-4o-mini-2024-07-18 != gpt-4o-mini. Model mapping may be incorrect.
  result = await self._model_client.create(
INFO:autogen_core:Publishing message of type GroupChatMessage to all subscribers: {'message': TextMessage(source='question_decomposition_agent', models_usage=RequestUsage(prompt_tokens=178, completion_tokens=20), content="[ { 'question': 'What is the total number of sales in 2008?', } ]")}
INFO:autogen_core:Publishing message of type GroupChatAgentResponse to all subscribers: {'agent_response': Response(chat_message=TextMessage(source='question_

---------- question_decomposition_agent ----------
[ { 'question': 'What is the total number of sales in 2008?', } ]
[Prompt tokens: 178, Completion tokens: 20]
---------- Summary ----------
Number of messages: 3
Finish reason: None
Total prompt tokens: 178
Total completion tokens: 20
Duration: 2.31 seconds


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='What are the total number of sales within 2008?'), TextMessage(source='sql_query_cache_agent', models_usage=None, content='{"contains_pre_run_results": false, "cached_questions_and_schemas": null}'), TextMessage(source='question_decomposition_agent', models_usage=RequestUsage(prompt_tokens=178, completion_tokens=20), content="[ { 'question': 'What is the total number of sales in 2008?', } ]")], stop_reason=None)